In [3]:
sqlContext

In [4]:
parquet = sqlContext.read.parquet(
    "hdfs://ip-172-31-27-198/user/ubuntu/day_BCSD_historical_r1i1p1_IPSL-CM5A-LR_1997.parquet")
parquet.registerTempTable("parquet")


In [5]:
parquet.show()

+-------+-----+---------+------------+---+---------+---------+
|    lat|  lon|     time|       model| pr|   tasmin|   tasmax|
+-------+-----+---------+------------+---+---------+---------+
|-89.625|0.375|849096000|IPSL-CM5A-LR|0.0|  271.062|273.07544|
|-89.625|0.625|849096000|IPSL-CM5A-LR|0.0|271.07523|273.08545|
|-89.625|0.875|849096000|IPSL-CM5A-LR|0.0| 271.0985|273.05838|
|-89.625|1.125|849096000|IPSL-CM5A-LR|0.0|271.10892|273.04404|
|-89.625|1.375|849096000|IPSL-CM5A-LR|0.0|271.11066|273.04037|
|-89.625|1.625|849096000|IPSL-CM5A-LR|0.0| 271.1192|273.00937|
|-89.625|1.875|849096000|IPSL-CM5A-LR|0.0|271.12167|272.97113|
|-89.625|2.125|849096000|IPSL-CM5A-LR|0.0|271.09567|272.96274|
|-89.625|2.375|849096000|IPSL-CM5A-LR|0.0|271.03177| 272.9883|
|-89.625|2.625|849096000|IPSL-CM5A-LR|0.0|270.97952|273.00665|
|-89.625|2.875|849096000|IPSL-CM5A-LR|0.0|270.96497|273.01193|
|-89.625|3.125|849096000|IPSL-CM5A-LR|0.0|270.94412|273.02158|
|-89.625|3.375|849096000|IPSL-CM5A-LR|0.0|270.91867|273

In [50]:
parquet_count = parquet.count()
parquet_count

376609324

### Create Dataframe
Pull out 'month' and 'year' from timestamp and make them available as columns.  Also exclude any datapoints that do not have valid values for pr, tasmin or tasmax.

In [6]:
sql = """
SELECT lat, lon, time, model, pr, tasmin, tasmax, MONTH(from_unixtime(time)) as month, YEAR(from_unixtime(time)) as year
FROM parquet
WHERE pr < 1.0E20 AND tasmin < 1.0E20 AND tasmax < 1.0E20
"""
df = sqlContext.sql(sql)

In [7]:
df.show()

+-------+-----+---------+------------+---+---------+---------+-----+----+
|    lat|  lon|     time|       model| pr|   tasmin|   tasmax|month|year|
+-------+-----+---------+------------+---+---------+---------+-----+----+
|-89.625|0.375|849096000|IPSL-CM5A-LR|0.0|  271.062|273.07544|   11|1996|
|-89.625|0.625|849096000|IPSL-CM5A-LR|0.0|271.07523|273.08545|   11|1996|
|-89.625|0.875|849096000|IPSL-CM5A-LR|0.0| 271.0985|273.05838|   11|1996|
|-89.625|1.125|849096000|IPSL-CM5A-LR|0.0|271.10892|273.04404|   11|1996|
|-89.625|1.375|849096000|IPSL-CM5A-LR|0.0|271.11066|273.04037|   11|1996|
|-89.625|1.625|849096000|IPSL-CM5A-LR|0.0| 271.1192|273.00937|   11|1996|
|-89.625|1.875|849096000|IPSL-CM5A-LR|0.0|271.12167|272.97113|   11|1996|
|-89.625|2.125|849096000|IPSL-CM5A-LR|0.0|271.09567|272.96274|   11|1996|
|-89.625|2.375|849096000|IPSL-CM5A-LR|0.0|271.03177| 272.9883|   11|1996|
|-89.625|2.625|849096000|IPSL-CM5A-LR|0.0|270.97952|273.00665|   11|1996|
|-89.625|2.875|849096000|IPSL-CM5A-LR|

In [51]:
df_count = df.count()
df_count

376605004

Percent of cells excluded due to missing values

In [54]:
((df_count - parquet_count) / float(parquet_count)) * 100

-0.001147077282664409

Make sure we've actually removed missing values (encoded as 1.0E20)

In [8]:
df.select("pr", "tasmin", "tasmax").agg({"pr": "max", 
                                     "tasmin": "max", 
                                     "tasmax": "max"}).show()

+-----------+-----------+------------+
|max(tasmin)|max(tasmax)|     max(pr)|
+-----------+-----------+------------+
|  310.95816|  322.88812|0.0056666597|
+-----------+-----------+------------+



Growing region is assumed to be April 1st through October 31st in the Northern Hemisphere,  and October 1st through April 30th in the Southern Hemisphere

In [11]:
grow_season = df.select("*")\
              .where(((df.lat >= 0.0) & (df.month >= 4) & (df.month <= 10)) |
                     ((df.lat < 0.0) & (df.month <= 4) & (df.month >= 10)))

In [15]:
gs_count = grow_season.count()
gs_count

110856640

Percent decrease of data due to filtering on grow season

In [16]:
((gs_count - df_count ) / float(df_count)) * 100

-70.56421480793708

In [25]:
grow_season.persist()

DataFrame[lat: float, lon: float, time: bigint, model: string, pr: float, tasmin: float, tasmax: float, month: int, year: int]

In [30]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, FloatType

def _k_to_f(k):
    return ((k - 273.15) * 1.8) + 32.0

def _avg_tmp_f(_min, _max):
    return (_k_to_f(_min) + _k_to_f(_max)) / 2.0

avg_tmp_f = udf(_avg_tmp_f, FloatType())

In [26]:
grow_season = grow_season.withColumn('f_tasavg', avg_tmp_f(df.tasmin, df.tasmax))
grow_season.show()

+-----+-----+---------+------------+------------+---------+---------+-----+----+---------+
|  lat|  lon|     time|       model|          pr|   tasmin|   tasmax|month|year| f_tasavg|
+-----+-----+---------+------------+------------+---------+---------+-----+----+---------+
|0.125|0.375|859896000|IPSL-CM5A-LR| 7.648604E-5|293.99588|294.51733|    4|1997| 69.99189|
|0.125|0.625|859896000|IPSL-CM5A-LR| 7.515501E-5| 293.9915|294.52817|    4|1997| 69.99769|
|0.125|0.875|859896000|IPSL-CM5A-LR| 7.382399E-5| 293.9925|294.52426|    4|1997| 69.99508|
|0.125|1.125|859896000|IPSL-CM5A-LR|  4.55957E-5| 293.9917|294.52222|    4|1997| 69.99252|
|0.125|1.375|859896000|IPSL-CM5A-LR|4.4758526E-5| 293.9889|294.52194|    4|1997| 69.98975|
|0.125|1.625|859896000|IPSL-CM5A-LR|4.3921355E-5| 293.9902|294.50815|    4|1997|69.978516|
|0.125|1.875|859896000|IPSL-CM5A-LR|4.3084186E-5|293.99295|294.48712|    4|1997| 69.96207|
|0.125|2.125|859896000|IPSL-CM5A-LR|4.5778543E-5|293.98972| 294.4758|    4|1997| 69.94897|

In [29]:
grow_season.where(grow_season.f_tasavg > 50.0).count()

60415559

In [37]:
# https://en.wikipedia.org/wiki/Winkler_scale
def _winkler_scale(d):
    if d <= 2500:
        return 1
    elif d >= 2501 and d <= 3000:
        return 2
    elif d >= 3001 and d <= 3500:
        return 3
    elif d >= 3501 and d <= 4000:
        return 4
    elif d > 4000:
        return 5

def _degree_days(temp):
    dd = int(temp - 50.0)
    return 0 if dd <= 0 else dd

degree_days = udf(_degree_days, IntegerType())
winkler_scale = udf(_winkler_scale, IntegerType())

### Calculate the degree days
Group By year, latitude, longitude and sum the calculated dgree days

In [38]:
dd = grow_season.withColumn("degree_days", degree_days(grow_season.f_tasavg))\
        .groupBy(df.year, df.lat, df.lon).agg({"degree_days": "sum"})\
        .withColumnRenamed("sum(degree_days)", "degree_days")

Exclude locations with less than 1 degree day

In [56]:
DEGREE_DAY_THRESHOLD = 1
dd = dd.where(dd.degree_days >= DEGREE_DAY_THRESHOLD)

In [57]:
dd.show()

+----+-----+-------+-----------+
|year|  lat|    lon|degree_days|
+----+-----+-------+-----------+
|1997|0.125|  2.125|       3200|
|1997|0.125|  8.125|       5863|
|1997|0.125| 14.375|       5902|
|1997|0.125| 22.125|       6335|
|1997|0.125| 28.375|       5681|
|1997|0.125| 34.125|       5461|
|1997|0.125| 40.375|       7361|
|1997|0.125| 46.625|       3696|
|1997|0.125| 52.875|       3740|
|1997|0.125| 59.125|       3986|
|1997|0.125| 69.875|       4132|
|1997|0.125| 76.125|       4175|
|1997|0.125| 82.375|       4194|
|1997|0.125| 88.625|       4203|
|1997|0.125| 94.875|       4289|
|1997|0.125|101.125|       6938|
|1997|0.125|107.375|       6696|
|1997|0.125|113.625|       5528|
|1997|0.125|119.875|       6099|
|1997|0.125|126.125|       6618|
+----+-----+-------+-----------+
only showing top 20 rows



In [58]:
dd.count()

363817

In [59]:
pdd = dd.withColumn("winkler", winkler_scale(dd.degree_days)).toPandas()

In [60]:
len(pdd)

363817

In [61]:
pdd

,year,lat,lon,degree_days,winkler
0,1997,3.125,96.375,6653,5
1,1997,3.125,102.625,6602,5
2,1997,3.125,108.875,6806,5
3,1997,3.125,115.125,4584,5
4,1997,3.125,121.375,4942,5
5,1997,3.125,127.625,6628,5
6,1997,3.125,130.375,5007,5
7,1997,3.125,136.625,4475,5
8,1997,3.125,142.875,4467,5
9,1997,3.125,149.125,4389,5


In [62]:
pdd.groupby("winkler").size()

winkler
1    157663
2     19846
3     22979
4     41634
5    121695
dtype: int64

In [63]:
pdd.to_csv("/home/ubuntu/winkler_scale_IPSL-CM5A-LR_1997.csv")